In [1]:
import osmnx as ox # open street map API
import ee # GEE API
import geopandas as gpd # geopandas

import json
import numpy as np
import pandas as pd
from shapely.geometry import Point
import rasterio
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
pd.set_option("display.max_rows", 10000)

In [3]:
ee.Initialize(project='cs231n-457303')

In [4]:
OUTPUT_PATH = r"C:\Users\renee\Documents\cs231n\solarTestFiles\outputs"

## Helper functions

In [5]:
# Dont actually use this as of now but kept around just in case
# Compute relative direction for each matched pair
def get_dir(row):
    x1, y1 = row.geometry.coords[0]          # building centroid
    x2, y2 = gdf_lot_pts.loc[row.lot_idx].geometry.coords[0]  # lot centroid

    dx = x1 - x2
    dy = y1 - y2
    angle = (np.degrees(np.arctan2(dy, dx)) + 360) % 360

    if 135 <= angle <= 225:
        return "south"
    elif 45 <= angle < 135:
        return "east"
    elif 225 < angle <= 315:
        return "west"
    else:
        return "north"

## Automatic
## pull data for a region

In [6]:
tags = {"amenity": "parking"}
region = "Los Angeles, California"
gdf_all = ox.features_from_place(region, tags=tags)
print(len(gdf_all))
# drop parking lots that only have centorid (not bounding box)
gdf_all = gdf_all.loc[gdf_all.index.get_level_values(0).isin(['way', 'relation'])]
print(len(gdf_all))
gdf_poly = gdf_all[gdf_all.geom_type.isin(["Polygon", "MultiPolygon"])]
gdf_poly = gdf_poly.reset_index(drop=False)
print(len(gdf_poly))

17162
16941
16941


## export shapefiles

In [53]:
png_to_coords = dict()

# Export as shapefile
for i in range(10000, len(gdf_poly)):
    path = os.path.join(OUTPUT_PATH, f"shapefiles\lot{i}")
    png_name = f"lot{i}.png"
    lot = gdf_poly.iloc[i].geometry
    y = lot.centroid.y
    x = lot.centroid.x
    lot = gpd.GeoDataFrame(geometry=[lot], crs="EPSG:4326")
    lot.to_file(path, driver="ESRI Shapefile")
    lot_metric = lot.to_crs("EPSG:3857")
    area_m2 = lot_metric.geometry[0].area  # in square meters
    png_to_coords[png_name] = {"lat": y, "lon": x, "area": area_m2}

# Save to JSON file
output_path = os.path.join(OUTPUT_PATH, "png_to_coords.json")
with open(output_path, "w") as f:
    json.dump(png_to_coords, f, indent=2)

print(f"Saved JSON to {output_path}")

Saved JSON to C:\Users\renee\Documents\cs231n\solarTestFiles\outputs\png_to_coords.json


In [ ]:
# Initialize GEE
ee.Initialize(project='cs231n-457303')
print(ee.data.getAssetRoots())

[{'type': 'Folder', 'id': 'projects/cs231n-457303/assets/folder_of_assets'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot0'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot1'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot2'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot3'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot4'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile1'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile10'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile2'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile3'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile4'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile5'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile6'}, {'type': 'Table', 'id': 'projects/cs231n-457303/assets/lot_shapefile7'}, {'type': 'Table', 'id':

In [ ]:
for i in range(16000, 16941):
    # Load shapefile and get geometry
    geom = gpd.read_file(f"{OUTPUT_PATH}\shapefiles\lot{i}\lot{i}.shp")
    geom = geom.iloc[0].geometry.__geo_interface__  # GeoJSON-style dict
    geom = ee.Geometry(geom)
    geom = geom.buffer(10)

    # get naip image
    naip = ee.ImageCollection("USDA/NAIP/DOQQ") \
        .filterBounds(geom) \
        .filterDate("2018-01-01", "2023-12-31")
    naip = naip.mosaic().clip(geom)
    naip = naip.visualize(
        bands=['R', 'G', 'B'],
        min=0,
        max=255
    )

    # send task
    task = ee.batch.Export.image.toDrive(
        image=naip,
        description=f'lot{i}',
        folder='earthengine2',
        fileNamePrefix=f'lot{i}',
        region=geom.bounds().getInfo()['coordinates'],
        scale=0.6,
        crs='EPSG:4326',
        maxPixels=1e9,
    )
    task.start()

In [ ]:
print(task.status())

{'state': 'COMPLETED', 'description': 'lot16940', 'priority': 100, 'creation_timestamp_ms': 1749015566286, 'update_timestamp_ms': 1749020117890, 'start_timestamp_ms': 1749020106342, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1yF9o7XQtlp6DXeU3IwHbKZysDeJTZn9i'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.3071926236152649, 'id': 'VAE4BK762H2WHOBWIY7QEFYV', 'name': 'projects/cs231n-457303/operations/VAE4BK762H2WHOBWIY7QEFYV'}


In [103]:
for i in range(16000, len(gdf_poly)):
    tif_path = os.path.join(OUTPUT_PATH, f"tifs\lot{i}.tif")

    # Load TIF
    with rasterio.open(tif_path) as src:
        array = src.read([1, 2, 3])  # RGB bands
        array = np.transpose(array, (1, 2, 0))  # CHW -> HWC

    array = array.astype(np.float32)
    array = 255 * (array - array.min()) / (array.max() - array.min())
    array = array.astype(np.uint8)

    # Save to PNG
    img_path = os.path.join(OUTPUT_PATH, f"pngs/lot{i}.png")
    Image.fromarray(array).save(img_path)

In [ ]:
import requests
import time

with open(os.path.join(OUTPUT_PATH, "png_to_coords.json"), "r") as f:
    png_coords = json.load(f)

url = "https://developer.nrel.gov/api/solar/solar_resource/v1.json"
api_key = "u3hOthZll5RD2nYBa2isJRKzfM1JQ99wEA8D1eQM"

solar_data = {}
from itertools import islice
first_1000 = dict(islice(png_coords.items(), min(1000, len(png_coords))))

with open(os.path.join(OUTPUT_PATH, "solar_irradiance_results.json"), "r") as f:
    solar_data = json.load(f)

for png_name, coords in first_1000.items():
    lon = coords["lon"]
    lat = coords["lat"]
    area = coords["area"]

    params = {
    "api_key": api_key,
    "lat": lat,
    "lon": lon
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            annual_irrad = data['outputs']['avg_dni']['annual']  # Direct Normal Irradiance (DNI)
            solar_data[png_name] = {
                "lat": lat,
                "lon": lon,
                "area": area,
                "annual_dni_kWh/m2/day": annual_irrad
            }
        else:
            print(f"Failed for {png_name}: {response.status_code}")
            solar_data[png_name] = {"error": f"Status code {response.status_code}"}
    except Exception as e:
        print(f"Error for {png_name}: {e}")
        solar_data[png_name] = {"error": str(e)}

    time.sleep(0.1)  # be polite to the API

# Save results
with open(os.path.join(OUTPUT_PATH, "solar_irradiance_results.json"), "w") as f:
    json.dump(solar_data, f, indent=2)

print("Finished fetching solar irradiance data.")

Finished fetching solar irradiance data.
